In [ ]:
import matplotlib
matplotlib.use("PDF")
from pylab import *
import sys, glob, xlrd, datetime
from scipy import stats
thedir='/Users/cslage/Research/LSST/optical_simulator/timing/ltspice/sta3800_output_device'
%cd $thedir


In [ ]:
data_wb = xlrd.open_workbook('STA3800_meas.xls')
idvd1_data = data_wb.sheet_by_name('8-1010.TXT')
idvd2_data = data_wb.sheet_by_name('8-1015.TXT')
idvg1_data = data_wb.sheet_by_name('8-1011.TXT')
idvg2_data = data_wb.sheet_by_name('8-1013.TXT')
idvg3_data = data_wb.sheet_by_name('8-1014.TXT')
cv_data = data_wb.sheet_by_name('C-V')


In [ ]:
def MOS_I(Vgs, Vds):
    Vt = -19.76
    Id0 = 0.0183
    Ioffset = 0.0
    if Vgs < Vt:
        return 0.0
    elif Vds < (Vgs-Vt):
        return Id0 * ((Vgs - Vt) * Vds - Vds**2 / 2.0) + Ioffset
    else:
        Vdsat = Vgs - Vt
        return Id0 * ((Vgs - Vt) * Vdsat - Vdsat**2 / 2.0) + Ioffset


In [ ]:
# ID-VD 2
W= 28.0
L = 5.0
Mu = 1000.0
Vbs = 0.0
Vgs_start = -16.0
Vgs_step = 4.0
Vgs_num_steps = 8
Vds_num_steps = 251
Vds = zeros(Vds_num_steps)
Vgs = linspace(Vgs_start, Vgs_start + Vgs_step * (Vgs_num_steps - 1), Vgs_num_steps)
Ids = zeros([Vgs_num_steps, Vds_num_steps])
for i in range(idvd2_data.nrows):
    try:
        if type(idvd2_data.row(i)[0].value) is float:
            vds_index = (int(idvd2_data.row(i)[0].value) - 1) % Vds_num_steps
            vgs_index = int(round((idvd2_data.row(i)[1].value - Vgs_start) / Vgs_step))
            if vgs_index == 0:
                Vds[vds_index] = idvd2_data.row(i)[2].value
            Ids[vgs_index, vds_index] = idvd2_data.row(i)[3].value
    except:
        continue

fig = figure()
ax1=axes([0.2,0.1,0.6,0.6])
ax1.set_title("STA3800C Id-Vd")
for i in range(Vgs_num_steps-1, 1, -1):
    ax1.plot(Vds, Ids[i,:]*1000.0, label = 'Vgs = %.1f V'%Vgs[i], lw = 2)
    for j in range(0,251,10):
        #print Vgs[i], Vds[j], MOS_I(Vgs[i], Vds[j])
        ax1.scatter(Vds[j], MOS_I(Vgs[i], Vds[j]))
ax1.set_xlabel("Vds (volts)")
ax1.set_ylabel("Ids(mA)")
ax1.set_ylim(0,8.0)
ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize = 6)
show()
#savefig("IdVd1_8Mar17.pdf")

In [ ]:
clear()